In [214]:
import pandas as pd
import re
import json  #Asegurarse de importar la biblioteca json
import html

#Load the JSON file
file_path = '//wsl.localhost//Ubuntu//home//lourdes22//data-pipeline//sources//webscraper//files//Scrape_Result_2024-09-26-10-53-47.json'

def preprocessing_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    #Verificar si el archivo está vacío
    if not content.strip():
        print("El archivo está vacío o solo contiene espacios en blanco.")
    else:

        #Intentar extraer JSON solo desde donde parece comenzar
        
        cleaned_content_filtered = ''.join(char for char in content if char.isprintable())

        # Reemplazar comillas simples por comillas dobles para hacer el JSON válido
        cleaned_content_filtered = re.sub(r"(?<!\\)'", '"', cleaned_content_filtered)
        cleaned_content_filtered = re.sub(
    r'("summary":\s*".*?)(<a.*?href=".*?")(.*?)(</a>)(.*?)(, "summary_detail")', 
    r'\1\2\4\6', 
    cleaned_content_filtered
)
        # Expresión regular para eliminar todo lo que hay después del enlace hasta "source"
        cleaned_content_filtered = re.sub(
    r'("value":\s*".*?\?oc=5")(.*?)("source":)', 
    r'\1, \3', 
    cleaned_content_filtered
)       
        cleaned_content_filtered = re.sub(
        r'("title":\s*")(.*?)(")',  # Encuentra el campo title y el contenido entre comillas
        lambda match: match.group(1) + match.group(2).replace('"', '') + match.group(3),  # Elimina comillas internas dentro del valor
        cleaned_content_filtered
    )
        
        
        # Reemplazar el campo 'published_parsed' con una cadena vacía
        cleaned_content_filtered = re.sub(r'"published_parsed":\s*time\.struct_time\([^)]*\),', '"published_parsed": "N/A",', cleaned_content_filtered)
        json_start = cleaned_content_filtered.find('{')
        if json_start != -1:
            json_content = cleaned_content_filtered[json_start:]  # Intentar extraer desde donde comienza JSON
            

  # Reemplazar el campo 'published_parsed' con una cadena vacía
        cleaned_content_filtered = re.sub(r'"updated_parsed":\s*time\.struct_time\([^)]*\),', '"updated_parsed": "N/A",', cleaned_content_filtered)
        json_start = cleaned_content_filtered.find('{')
        if json_start != -1:
            json_content = cleaned_content_filtered[json_start:]  # Intentar extraer desde donde comienza JSON

            # Replace problematic values with valid JSON
            json_content = json_content.replace("True", "true").replace("False", "false")
            json_content = json_content.replace("None", "null")
            json_content = json_content.replace('"<a href="', '"')
            json_content = json_content.replace('</a>', '')
            json_content = json_content.replace('"Boxes"', 'Boxes')
            json_content = json_content.replace('"Normalize"', 'Normalize')
            json_content = json_content.replace('"s ', '\'s ')
            json_content = json_content.replace('s" ', 's\' ')
            json_content = json_content.replace(' "', ' \'')
            json_content = json_content.replace("'entries", '"entries')
            json_content = json_content.replace(": '", ': "')
            json_content = json_content.replace(", '", ', "')
            json_content = json_content.replace('" ', "' ")
            json_content = json_content.replace('""', '"')
            json_content = re.sub(r'"content-security-policy":\s*".*?",', '"content-security-policy": "",', json_content)

            
        # Ver la sección problemática en torno al carácter 25799
            problematic_section = json_content[81818     :84000    ]
            print("Problematic section:", problematic_section)
            try:
                #Convertir el contenido a un objeto JSON
                json_data = json.loads(json_content)
                print(json_data)
            
                #Extraer títulos de 'entries'
                titles = [entry.get('title', '') for entry in json_data.get('entries', [])]

                #Crear un DataFrame con los títulos
                df_titles = pd.DataFrame(titles, columns=['Title'])

                #Mostrar el DataFrame
                print(df_titles)

            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON después de la limpieza: {e}")

                #Intentar extracción con expresiones regulares como respaldo
                title_pattern = re.compile(r'"title":\s*"([^"]+)"')  #Patrón para encontrar los títulos en el texto
                titles = title_pattern.findall(cleaned_content_filtered)  #Extraer todos los títulos coincidentes
            
                #Crear un DataFrame con los títulos
                df_titles = pd.DataFrame(titles, columns=['Title'])
            
                #Mostrar el DataFrame
                return df_titles
        else:
            print("No se encontró un JSON válido en el contenido del archivo.")


preprocessing_json(file_path)

Problematic section: gin-allow-popups", "content-security-policy": "", "accept-ch": "Sec-CH-UA-Arch, Sec-CH-UA-Bitness, Sec-CH-UA-Full-Version, Sec-CH-UA-Full-Version-List, Sec-CH-UA-Model, Sec-CH-UA-WoW64, Sec-CH-UA-Form-Factors, Sec-CH-UA-Platform, Sec-CH-UA-Platform-Version", "reporting-endpoints": "default="/_/DotsSplashUi/web-reports?context=eJzjStDikmJw1ZBiUAjbySTx9SWTGhA7pc9gDQDiTztmsLbePMc6GYiT_p1nLQDiJREXWQ8kXmQ1VLjEag_El9Mvsar2XGI1BuJ6hsusTKaXWa3yL7MWSVxhbQBiIR6OI-eubmcTmPCodx2jknJSfmF8Sn5JcXFBTmJxRnFqUVlqUbyRgZGJgaWRsZ6hYXyBAQDmAziV", "content-encoding": "gzip", "server": "ESF", "x-xss-protection": "0", "x-frame-options": "SAMEORIGIN", "x-content-type-options": "nosniff", "date": "Thu, 26 Sep 2024 13:53:08 GMT", "expires": "Mon, 01 Jan 1990 00:00:00 GMT", "cache-control": "no-cache, no-store, max-age=0, must-revalidate", "content-type": "application/xml; charset=utf-8", "vary": "Sec-Fetch-Dest, Sec-Fetch-Mode, Sec-Fetch-Site", "content-length": "29916", "age": "39", "alt-svc

,Title
0,Will the US presidential election define the f...
1,BBC.com
2,7 Best Crypto to Buy and Invest Now In Septemb...
3,NFTevening.com
4,"Crypto investor turns $1,300 to $3.4M in 15 da..."
...,...
392,The Crypto Basic
393,"PayPal extends the buy, hold, and sell Bitcoin..."
394,The Cryptonomist
395,Dubai regulators enforce new rule that mandate...


In [63]:
import os
import pandas as pd
import re
import json
import datetime
import csv

# Ruta de las carpetas
webscraping_dir = '//wsl.localhost//Ubuntu//home//lourdes22//data-pipeline//sources//webscraper//files//'
preprocessed_dir = '//wsl.localhost//Ubuntu//home//lourdes22//data-pipeline//sources//webscraper//files//'

# Función para obtener el archivo JSON más reciente
def get_latest_json_file(directory):
    json_files = [f for f in os.listdir(directory) if f.endswith('.json')]
    if not json_files:
        print("No se encontraron archivos JSON en la carpeta.")
        return None

    # Ordenar archivos por fecha, basado en el nombre que contiene la fecha
    json_files.sort(key=lambda x: os.path.getmtime(os.path.join(directory, x)), reverse=True)
    latest_file = json_files[0]
    return os.path.join(directory, latest_file)

# Función para preprocesar el archivo JSON
def preprocessing_json_from_directory(directory, start_index=268):
    file_path = get_latest_json_file(directory)
    if not file_path:
        return
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            
            
        #Verificar si el archivo está vacío
        if not content.strip():
            print("El archivo está vacío o solo contiene espacios en blanco.")
        else:
        #Limpiar el contenido eliminando cualquier carácter no imprimible
            cleaned_content_filtered = ''.join(char for char in content if char.isprintable())

        #Reemplazar comillas simples por comillas dobles para hacer el JSON válido
            cleaned_content_filtered = re.sub(r"(?<!\\)'", '"', cleaned_content_filtered)

        #Intentar extraer JSON solo desde donde parece comenzar
            json_start = cleaned_content_filtered.find('{')
            if json_start != -1:
                json_content = cleaned_content_filtered[json_start:]  #Intentar extraer desde donde comienza JSON
                print(json_content)
                try:
                    #Convertir el contenido a un objeto JSON
                    json_data = json.loads(json_content)
                    print(json_data)
                    #Extraer títulos de 'entries'
                    titles = [entry.get('title', '') for entry in json_data.get('entries', [])]
                    
            # Extraer los enlaces 'href' dentro de 'links'
                    href_links = []
                    for entry in json_data.get('entries', []):
                        if 'links' in entry:
                            print("t")
                 # Obtenemos el primer link con su 'href'
                            href = entry['links'][0].get('href', '')
                            print(href)
                            href_links.append(href)
                        else:
                            href_links.append('')  # Si no tiene links, añadimos un valor vacío

        # Crear un DataFrame con los títulos, fechas y enlaces
                        df = pd.DataFrame({
                    'Title': titles,
                        })

        # Mostrar el DataFrame
                    print(df)

                    return df
    

                except json.JSONDecodeError as e:
                    print(f"Error al decodificar JSON después de la limpieza: {e}")

    except json.JSONDecodeError as json_error:
        print(f"Error decoding JSON: {json_error}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Ejecutar la función
preprocessing_json_from_directory(webscraping_dir, start_index=268)


{"bozo": False, "entries": [{"title": "Will the US presidential election define the future of crypto? - BBC.com", "title_detail": {"type": "text/plain", "language": None, "base": "https://news.google.com/rss/search?q=crypto+when:1d&hl=en-US&gl=US&ceid=US:en", "value": "Will the US presidential election define the future of crypto? - BBC.com"}, "links": [{"rel": "alternate", "type": "text/html", "href": "https://news.google.com/rss/articles/CBMiWkFVX3lxTE4wWFh6OU5FMHlZWFJmZ2x5dFVOVzlnbzNBVllQSG9xUlBJMGM2LW9xTzI0RmlHTkctWGNBYU55eHg4WTAxLWlwc2hZWWtzbE5yMFdHVlZVUzVOd9IBX0FVX3lxTFBfQnJzLVVmMmVqS2YzVWlGNkxTREpFSEh6MFl3S2pnSEw5UkxtekFub2lmZWppY2xQd3h4Nnd1X1F5Y0tuS1JsSklIdGlmeVEyemFsbXNGNGZYc1RGOEs4?oc=5"}], "link": "https://news.google.com/rss/articles/CBMiWkFVX3lxTE4wWFh6OU5FMHlZWFJmZ2x5dFVOVzlnbzNBVllQSG9xUlBJMGM2LW9xTzI0RmlHTkctWGNBYU55eHg4WTAxLWlwc2hZWWtzbE5yMFdHVlZVUzVOd9IBX0FVX3lxTFBfQnJzLVVmMmVqS2YzVWlGNkxTREpFSEh6MFl3S2pnSEw5UkxtekFub2lmZWppY2xQd3h4Nnd1X1F5Y0tuS1JsSklIdGlmeVEyemFsbXNG